In [ ]:
# ===============================
# 0. 라이브러리 및 API 설정
# ===============================
import os
import json
from datetime import timedelta
import pandas as pd
import numpy as np

# 필요시 한 번만 설치 (주피터/코랩에서)
# !pip install google-generativeai python-docx openpyxl

import google.generativeai as genai
from docx import Document  # 워드 템플릿용

# ✅ Gemini API 키 설정 (본인 키로 바꿔 쓰세요)
os.environ["GEMINI_API_KEY"] = ""
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# 사용할 Gemini 모델
gemini_model = genai.GenerativeModel("gemini-1.5-pro")


# ===============================
# 0-1. 원료 투입 기록지 워드 템플릿 로드 (존재 확인)
# ===============================
template_path = r"C:\Users\user\Downloads\원료_투입_기록지.docx" 

if not os.path.exists(template_path):
    print(f"[WARN] 워드 템플릿을 찾을 수 없습니다: {template_path}")
    doc = None
else:
    doc = Document(template_path)
    print(f"[INFO] 워드 템플릿 로드 완료: {template_path}")
    print(f"[INFO] 템플릿 내 테이블 개수: {len(doc.tables)}")
    # 필요하면 나중에 doc.tables[...] 를 사용해서 표 채우기 가능


# ===============================
# 1. 입력 데이터 로드
# ===============================
input_path = r"C:\Users\user\Downloads\입력 데이터.xlsx"  # 환경에 맞게 수정 가능
df = pd.read_excel(input_path, header=0)

# 필수 컬럼 이름 체크
required_cols = {"time_sec", "flag_id"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"엑셀에 필요한 컬럼 {missing} 이(가) 없습니다. 컬럼명을 확인해 주세요.")

# ⭕ time_sec 를 datetime 으로 변환 (이미 datetime 이면 그대로 유지)
df["time_sec"] = pd.to_datetime(df["time_sec"])

# ⭕ flag_id 숫자/문자 → A/S/D 로 정규화
def normalize_flag(x):
    if pd.isna(x):
        return None
    s = str(x).strip().upper()
    if s in ["1", "A"]:
        return "A"
    if s in ["2", "S"]:
        return "S"
    if s in ["3", "D"]:
        return "D"
    return None  # 그 외 값은 이벤트로 사용 안 함

df["flag_norm"] = df["flag_id"].apply(normalize_flag)

print("[DEBUG] flag_norm value_counts:")
print(df["flag_norm"].value_counts(dropna=False))

# 시간 기준 정렬
df = df.sort_values("time_sec").reset_index(drop=True)


# ===============================
# 2. 유틸 함수: datetime → 시각 문자열
# ===============================
def format_time(dt) -> str:
    """datetime/Timestamp 를 'HH:MM:SS' 형식 문자열로 변환"""
    return dt.strftime("%H:%M:%S")


# ===============================
# 3. A/S/D 이벤트 구간 묶기 (flag_norm 사용)
#   - 연속된 같은 flag_norm 를 하나의 이벤트로 간주
#   - None/기타 값은 이벤트로 취급하지 않음
# ===============================
events = []  # 각 요소: dict(idx, flag, start_time, end_time, duration_sec)

current_flag = None
current_start = None   # datetime
prev_time = None       # datetime
event_index = 0

for i, row in df.iterrows():
    t = row["time_sec"]        # datetime
    f = row["flag_norm"]       # "A"/"S"/"D"/None

    is_action = f in ["A", "S", "D"]

    if not is_action:
        # 현재 진행 중이던 이벤트가 있으면 종료
        if current_flag is not None:
            duration_sec = (prev_time - current_start).total_seconds()
            events.append(
                {
                    "idx": event_index,
                    "flag": current_flag,       # "A", "S", "D"
                    "start_time": current_start,
                    "end_time": prev_time,
                    "duration_sec": float(duration_sec),
                }
            )
            event_index += 1
            current_flag = None
            current_start = None
        prev_time = t
        continue

    # A/S/D 인 경우
    if current_flag is None:
        # 새 이벤트 시작
        current_flag = f
        current_start = t
    elif f != current_flag:
        # 다른 행동으로 바뀌면 이전 이벤트를 닫고 새 이벤트 시작
        duration_sec = (prev_time - current_start).total_seconds()
        events.append(
            {
                "idx": event_index,
                "flag": current_flag,
                "start_time": current_start,
                "end_time": prev_time,
                "duration_sec": float(duration_sec),
            }
        )
        event_index += 1
        current_flag = f
        current_start = t

    prev_time = t

# 마지막 이벤트가 열려 있으면 닫기
if current_flag is not None and current_start is not None and prev_time is not None:
    duration_sec = (prev_time - current_start).total_seconds()
    events.append(
        {
            "idx": event_index,
            "flag": current_flag,
            "start_time": current_start,
            "end_time": prev_time,
            "duration_sec": float(duration_sec),
        }
    )

events_df = pd.DataFrame(events)
print("=== 이벤트 구간 ===")
print(events_df)

# ⭕ 이벤트가 하나도 없을 때 안전 처리
if events_df.empty:
    print("[WARN] A/S/D 이벤트가 하나도 생성되지 않았습니다. 'flag_id' 값과 맵핑을 확인하세요.")

    auto_log_df = pd.DataFrame(columns=["시간(Time)", "감지된 행동(AI Event)", "적합 여부(Status)", "비고(Remarks)"])
    auto_log_df.to_csv("AI자동기록(3번).csv", index=False, encoding="utf-8-sig")

    anomaly_df = pd.DataFrame(columns=["시간(Time)", "이상유형(Event Type)", "상세 내용(Description)", "자동 조치(Action)", "담당자 확인(Check)"])
    anomaly_df.to_csv("이상이벤트로그(4번).csv", index=False, encoding="utf-8-sig")

    raise SystemExit("이벤트가 없어 이후 로직을 수행하지 않습니다.")


# ===============================
# 4. 5초 이하 이벤트 및 A→D(중간 S 누락) 탐지
# ===============================
SHORT_THRESHOLD = 5.0  # 5초

# 4-1. 5초 이하 이벤트
short_event_indices = set(
    events_df.loc[events_df["duration_sec"] <= SHORT_THRESHOLD, "idx"].tolist()
)

# 4-2. A → D 순서에서 S 누락 탐지
missing_process_pairs = []  # 각 요소: dict(a_event, d_event)

for i, ev in events_df.iterrows():
    if ev["flag"] != "A":
        continue
    # A 다음에 오는 첫 번째 다른 행동 찾기
    j = i + 1
    while j < len(events_df) and events_df.iloc[j]["flag"] == "A":
        j += 1
    if j >= len(events_df):
        continue

    next_ev = events_df.iloc[j]
    # 정상은 A → S → D 이지만, 여기서는 바로 D가 오면 S 누락으로 판단
    if next_ev["flag"] == "D":
        missing_process_pairs.append(
            {
                "a_event": ev.to_dict(),
                "d_event": next_ev.to_dict(),
            }
        )

# D 이벤트 중에서 "프로세스 누락"에 해당하는 것들 인덱스
missing_process_d_indices = set(
    int(pair["d_event"]["idx"]) for pair in missing_process_pairs
)


# ===============================
# 5. 3번: AI 자동기록(3번) 테이블 생성
# ===============================
auto_rows = []

for _, ev in events_df.iterrows():
    idx = int(ev["idx"])
    flag = ev["flag"]              # "A"/"S"/"D"
    start_time = ev["start_time"]  # datetime
    end_time = ev["end_time"]      # datetime

    time_str = f"{format_time(start_time)} ~ {format_time(end_time)}"

    status = "정상"
    remarks = ""

    # 5초 이하 → 적합 여부 '확인 요망'
    if idx in short_event_indices:
        status = "확인 요망"

    # A 후 D, S 누락 → D 이벤트에만 비고
    if idx in missing_process_d_indices:
        if remarks:
            remarks += "; "
        remarks += "작업 프로세스 확인 요망"

    auto_rows.append(
        {
            "시간(Time)": time_str,
            "감지된 행동(AI Event)": flag,
            "적합 여부(Status)": status,
            "비고(Remarks)": remarks,
        }
    )

auto_log_df = pd.DataFrame(auto_rows)
print("\n=== 3번 AI 자동기록 미리보기 ===")
print(auto_log_df.head())

# CSV로 저장 (3번)
auto_log_output_path = "AI자동기록(3번).csv"
auto_log_df.to_csv(auto_log_output_path, index=False, encoding="utf-8-sig")
print(f"\n[저장 완료] {auto_log_output_path}")


# ===============================
# 6. 4번: 이상 이벤트 리스트 만들기
# ===============================
anomaly_items = []

# 6-1. 5초 이하 이벤트
for _, ev in events_df.iterrows():
    idx = int(ev["idx"])
    if idx not in short_event_indices:
        continue

    start_time = ev["start_time"]
    end_time = ev["end_time"]

    anomaly_items.append(
        {
            "type": "short_duration",
            "flag": ev["flag"],  # "A"/"S"/"D"
            "start_time_str": format_time(start_time),
            "end_time_str": format_time(end_time),
            "duration_sec": float(ev["duration_sec"]),
        }
    )

# 6-2. A→D (S 누락) 이벤트
for pair in missing_process_pairs:
    a_ev = pair["a_event"]
    d_ev = pair["d_event"]

    anomaly_items.append(
        {
            "type": "missing_process",
            "prev_flag": a_ev["flag"],
            "next_flag": d_ev["flag"],
            "prev_start_time_str": format_time(a_ev["start_time"]),
            "prev_end_time_str": format_time(a_ev["end_time"]),
            "next_start_time_str": format_time(d_ev["start_time"]),
            "next_end_time_str": format_time(d_ev["end_time"]),
        }
    )

print("\n=== 이상 이벤트 개수 ===", len(anomaly_items))


# ===============================
# 7. Gemini 프롬프트 함수 정의
# ===============================
def build_anomaly_prompt(event_json: str) -> str:
    return f"""
너는 제약/바이오 제조 공정(GMP)의 품질 관리 담당자이다.
다음 이상 이벤트 정보를 보고 기록지에 들어갈 '상세 내용'과 '자동 조치' 문장을 한국어로 작성하라.

반드시 아래 JSON 형식으로만 답하라. 불필요한 말은 쓰지 않는다.

입력 이벤트 정보(예시 형식):
{event_json}

출력 형식 (JSON):
{{
  "description": "<상세 내용 1~2문장>",
  "action": "<자동 조치 1문장>"
}}

요구사항:
- 5초 이하 이벤트(type=short_duration)는
  - 어떤 행동(flag)이
  - 어느 시간대(시작~종료)에
  - 몇 초 정도로 짧게 발생했는지
  - 해당 시간대를 작업/영상/로그로 확인해야 한다는 취지로 작성하라.
- 행동 누락(type=missing_process)는
  - 원래 A→S→D 순서여야 하는데 A 후에 바로 D가 진행되었다는 점,
  - 중간 공정(S) 누락 가능성,
  - 해당 배치/시간대의 작업 프로세스를 확인해야 한다는 취지로 작성하라.
"""


# ===============================
# 8. 4번: 이상 이벤트 로그 테이블 생성
# ===============================
anomaly_rows = []

for item in anomaly_items:
    if item["type"] == "short_duration":
        time_str = f"{item['start_time_str']} ~ {item['end_time_str']}"
        event_type = "5초 이하 행동"
    else:
        # missing_process
        time_str = (
            f"{item['prev_start_time_str']} ~ {item['next_end_time_str']}"
        )
        event_type = "행동 순서 누락(A→D)"

    # Gemini를 통한 상세 내용/자동 조치 생성
    event_json = json.dumps(item, ensure_ascii=False)
    prompt_text = build_anomaly_prompt(event_json)

    try:
        response = gemini_model.generate_content(prompt_text)
        raw = response.text.strip()
    except Exception as e:
        print("[ERROR] Gemini 호출 중 오류:", e)
        raw = '{"description": "모델 호출 오류로 상세 내용을 생성하지 못했습니다.", "action": "해당 시간대 작업 로그 및 CCTV를 수동으로 확인한다."}'

    # 모델 응답을 JSON으로 파싱 시도
    try:
        parsed = json.loads(raw)
        description = parsed.get("description", "").strip()
        action = parsed.get("action", "").strip()
    except Exception:
        # JSON 파싱 실패 시, 전체를 상세 내용에 넣고 액션은 기본값
        description = raw.strip()
        action = "해당 시간대 작업 로그 및 CCTV를 확인한다."

    anomaly_rows.append(
        {
            "시간(Time)": time_str,
            "이상유형(Event Type)": event_type,
            "상세 내용(Description)": description,
            "자동 조치(Action)": action,
            "담당자 확인(Check)": "",
        }
    )

anomaly_df = pd.DataFrame(anomaly_rows)
print("\n=== 4번 이상 이벤트 로그 미리보기 ===")
print(anomaly_df.head())

# CSV로 저장 (4번)
anomaly_output_path = "이상이벤트로그(4번).csv"
anomaly_df.to_csv(anomaly_output_path, index=False, encoding="utf-8-sig")
print(f"\n[저장 완료] {anomaly_output_path}")


[INFO] 워드 템플릿 로드 완료: C:\Users\user\Downloads\원료_투입_기록지.docx
[INFO] 템플릿 내 테이블 개수: 4
[DEBUG] flag_norm value_counts:
flag_norm
A    6
D    6
S    4
Name: count, dtype: int64
=== 이벤트 구간 ===
   idx flag              start_time                end_time  duration_sec
0    0    A 2025-11-21 13:41:00.960 2025-11-21 13:41:05.280         4.320
1    1    S 2025-11-21 13:41:06.144 2025-11-21 13:41:17.376        11.232
2    2    D 2025-11-21 13:41:17.376 2025-11-21 13:41:26.016         8.640
3    3    A 2025-11-21 13:41:35.520 2025-11-21 13:41:38.976         3.456
4    4    D 2025-11-21 13:41:48.480 2025-11-21 13:41:52.800         4.320
5    5    A 2025-11-21 13:42:10.944 2025-11-21 13:42:16.992         6.048
6    6    S 2025-11-21 13:42:18.720 2025-11-21 13:42:36.864        18.144
7    7    D 2025-11-21 13:42:39.456 2025-11-21 13:42:53.280        13.824

=== 3번 AI 자동기록 미리보기 ===
              시간(Time) 감지된 행동(AI Event) 적합 여부(Status)    비고(Remarks)
0  13:41:00 ~ 13:41:05                A         확인 요망

In [12]:
# ===============================
# 9. 3번 / 4번 CSV를 워드 템플릿에 채워넣기
# ===============================
from docx import Document

# 3번, 4번 CSV 로드
auto_log_df = pd.read_csv("AI자동기록(3번).csv")        # 시간, 감지된 행동, 적합 여부, 비고
anomaly_df  = pd.read_csv("이상이벤트로그(4번).csv")     # 시간, 이상유형, 상세 내용, 자동 조치, 담당자 확인

# 원본 템플릿 로드
template_path = r"C:\Users\user\Downloads\원료_투입_기록지.docx"
doc = Document(template_path)
print(f"[INFO] 템플릿 내 테이블 개수: {len(doc.tables)}")

# 특정 헤더 텍스트를 보고 테이블 인덱스를 찾는 유틸
def find_table_index_by_keyword(document, keyword: str):
    keyword = str(keyword)
    for idx, tbl in enumerate(document.tables):
        # 표의 앞쪽 1~2줄만 모아서 텍스트 검색
        header_text = "\n".join(
            " | ".join(cell.text for cell in row.cells)
            for row in tbl.rows[:2]
        )
        if keyword in header_text:
            return idx
    return None

# 3번: AI 자동기록용 테이블 찾기
ai_table_idx = find_table_index_by_keyword(doc, "감지된 행동")
if ai_table_idx is None:
    print("[WARN] '감지된 행동' 헤더를 가진 테이블을 찾지 못했습니다. 임시로 3번째 테이블(인덱스 2)을 사용합니다.")
    ai_table_idx = 2  # 필요시 조정
ai_table = doc.tables[ai_table_idx]
print(f"[INFO] AI 자동기록 테이블 index = {ai_table_idx}")

# 4번: 이상 이벤트 로그용 테이블 찾기
anomaly_table_idx = find_table_index_by_keyword(doc, "이상유형")
if anomaly_table_idx is None:
    print("[WARN] '이상유형' 헤더를 가진 테이블을 찾지 못했습니다. 임시로 4번째 테이블(인덱스 3)을 사용합니다.")
    anomaly_table_idx = 3  # 필요시 조정
anomaly_table = doc.tables[anomaly_table_idx]
print(f"[INFO] 이상 이벤트 로그 테이블 index = {anomaly_table_idx}")

# 공통 유틸: DataFrame 내용을 표에 채워넣기
def fill_table_from_df(table, df, col_map):
    """
    table : python-docx Table 객체
    df    : 채워넣을 pandas DataFrame
    col_map : [(테이블열인덱스, df컬럼이름), ...] 순서
    """
    # 헤더(0번 row)는 그대로 두고, 데이터는 1번 row부터 채움
    header_row_idx = 0
    required_rows = len(df) + 1  # 헤더 + 데이터 행 수

    # 필요한 만큼 행 추가
    while len(table.rows) < required_rows:
        table.add_row()

    # 각 행 채우기
    for i, (_, row) in enumerate(df.iterrows(), start=1):
        for col_idx, df_col in col_map:
            value = row.get(df_col, "")
            table.rows[i].cells[col_idx].text = "" if pd.isna(value) else str(value)

# ===============================
# 9-1. 3번: AI 자동기록 테이블 채우기
# ===============================
# 워드 테이블 컬럼 순서 예 assumed:
# 0: 시간(Time), 1: 감지된 행동(AI Event), 2: 적합 여부(Status), 3: 비고(Remarks)
ai_col_map = [
    (0, "시간(Time)"),
    (1, "감지된 행동(AI Event)"),
    (2, "적합 여부(Status)"),
    (3, "비고(Remarks)"),
]

fill_table_from_df(ai_table, auto_log_df, ai_col_map)
print("[INFO] 3번 AI 자동기록 테이블 채우기 완료.")

# ===============================
# 9-2. 4번: 이상 이벤트 로그 테이블 채우기
# ===============================
# 워드 테이블 컬럼 순서 예 assumed:
# 0: 시간(Time), 1: 이상유형(Event Type), 2: 상세 내용(Description),
# 3: 자동 조치(Action), 4: 담당자 확인(Check)
anomaly_col_map = [
    (0, "시간(Time)"),
    (1, "이상유형(Event Type)"),
    (2, "상세 내용(Description)"),
    (3, "자동 조치(Action)"),
    (4, "담당자 확인(Check)"),
]

fill_table_from_df(anomaly_table, anomaly_df, anomaly_col_map)
print("[INFO] 4번 이상 이벤트 로그 테이블 채우기 완료.")

# ===============================
# 9-3. 작성본 문서 저장
# ===============================
output_docx_path = r"C:\Users\user\Downloads\원료_투입_기록지_작성본.docx"
doc.save(output_docx_path)
print(f"[저장 완료] {output_docx_path}")


[INFO] 템플릿 내 테이블 개수: 4
[INFO] AI 자동기록 테이블 index = 2
[INFO] 이상 이벤트 로그 테이블 index = 3
[INFO] 3번 AI 자동기록 테이블 채우기 완료.
[INFO] 4번 이상 이벤트 로그 테이블 채우기 완료.
[저장 완료] C:\Users\user\Downloads\원료_투입_기록지_작성본.docx
